In [1]:
import cv2 as cv
import numpy as np
#import matplotlib.pyplot as plt

def canny(image):
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    canny = cv.Canny(gray,50,150)
    return canny

def mask(image):
    #h = 630
    h = image.shape[0]
    triangle = np.array([[(200, h),(1100, h), (550, 250)]])
    #triangle = np.array([[(210, h),(1280, h), (600, 410)]])
    mask = np.zeros_like(image)
    cv.fillPoly(mask, triangle, 255)
    masked = cv.bitwise_and(mask, image)
    return masked

def make_coordinates(image, line):
    slope_n, intercept_n = line
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept_n)/(slope_n))
    x2 = int((y2 - intercept_n)/(slope_n))
    return np.array([x1, x2, y1, y2])

        
   
def find_lines(image):
    lines = cv.HoughLinesP(masked,2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 200)
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        param = np.polyfit((x1,x2), (y1,y2), 1)
        slope = param[0]
        intercept = param[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
            
    avrg_left = np.average(left_fit, axis = 0)
    avrg_right = np.average(right_fit, axis = 0)
    
    left_line = make_coordinates(image, avrg_left)
    right_line = make_coordinates(image, avrg_right)
    
    return np.array([left_line, right_line])


def display_line_image(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1, x2), (y1, y2), (0, 255, 0), 5)
    return line_image        

def display_rect_image(image, lines):
    rect_image = np.zeros_like(image)
    if lines is not None:
        x1, x2, y1, y2 = lines[0].reshape(4)
        x3, x4, y3, y4 = lines[1].reshape(4)
        h = image.shape[0]
        h2 = int(h*(3/5))
        poly = np.array([[(x1, y1),(x2, y2), (x4, y4), (x3, y3)]])
        cv.fillPoly(rect_image, poly, (255,0,0))
        rec_image = cv.bitwise_and(rect_image, image)

    return rec_image        

def display_combo(image, line, rectangle):
    combo_one = cv.addWeighted(image, 0.8, line, 1, 1)
    combo_image = cv.addWeighted(combo_one, 0.8, rectangle, 1, 1)
    return combo_image


vid = cv.VideoCapture("videos/vid.mp4")
while vid.isOpened():
    _, frame = vid.read()
    canny_image = canny(frame)
    masked = mask(canny_image)
    avrg_lines = find_lines(masked)
    #print(avrg_lines)
    lines_image = display_line_image(frame, avrg_lines)
    r_image = display_rect_image(frame, avrg_lines)
    combo_image = display_combo(frame, lines_image, r_image)
    cv.imshow('video', combo_image)
    if cv.waitKey(10)&0xFF == ord('q'):
        break
vid.release()
cv.destroyAllWindows()